# Selecting spectral lines

In [ ]:
import pandas as pd
import spectrograph as sg
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
#from colorspacious import cspace_converter
import numpy as np
import warnings
warnings.filterwarnings('ignore')

In [ ]:
%matplotlib widget
%load_ext autoreload
%autoreload 2

In [ ]:
data = sg.CCD2d(**sg.default_kwargs)
cmap = cm.get_cmap('inferno')
colors = cmap(np.linspace(0,1, data.get_orders().shape[0]))
orders = data.get_orders()

We now write all order as a single function $x = f(\lambda o)$

In [ ]:
plt.figure(figsize=(5,4))
for o in orders:
    I = data.index_order(o)
    plt.plot(data.o[I]*data.l[I], data.x[I], 'o', color=colors[o-orders[0]])
plt.show()

Removing a first global polynomial of low order..

In [ ]:
ol = data.o * data.l
global_fit = np.polynomial.Polynomial.fit(sg.link_ol(ol), data.x, deg=4)
# residuals
res_1 = data.x - global_fit(sg.link_ol(ol))
# plotting the residuum
plt.figure(figsize=(4,3))
for o in orders:
    I = data.index_order(o)
    plt.plot(ol[I], res_1[I], '.', color=colors[o-orders[0]])
plt.show()

reducing by linear $o$ dependency
$$
x = P_1(ol)+P_2(o)
$$
Degree of $P_1$ is $4$, degree of $P_2$ is $2$.

In [ ]:
fit_polynomial2 = np.polynomial.Polynomial.fit(data.o, res_1, deg=2)
res_2 = res_1 - fit_polynomial2(data.o)
# plotting the residuum
plt.figure(figsize=(4,3))
for o in orders:
    I = data.index_order(o)
    plt.plot(ol[I], res_2[I], '.', color=colors[o-orders[0]])
    
# single out the strange order....(plotted in cyan)
o = 54
I = data.index_order(o)
plt.plot(ol[I], res_2[I], '.', color='cyan')
plt.show()

In [ ]:
# new set of orders
orders = np.array([23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39,
       40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 55, 56,
       57, 58])
plt.figure(figsize=(4,3))
for o in orders:
    I = data.index_order(o)
    plt.plot(ol[I], res_2[I], '.', color=colors[o-orders[0]])

refitting all with order 54 removed

In [ ]:
global_fit = np.polynomial.Polynomial.fit(sg.link_ol(ol), data.x, deg=4)
res_1 = data.x - global_fit(sg.link_ol(ol))
fit_polynomial2 = np.polynomial.Polynomial.fit(data.o, res_1, deg=3)
res_2 = res_1 - fit_polynomial2(data.o)
fit_polynomial3 = np.polynomial.Polynomial.fit(sg.link_ol(ol)*data.o, res_2, deg=2)
res_3 = res_2 - fit_polynomial3(sg.link_ol(ol))
plt.figure(figsize=(4,3))
for o in orders:
    I = data.index_order(o)
    plt.plot(ol[I], res_3[I], '.', color=colors[o-orders[0]])

In [ ]:
o = 57   # play with me....
I = data.index_order(o)
poly = np.polynomial.Polynomial.fit(sg.link_ol(ol[I]), res_3[I], deg = 6)
res_4 = res_3[I] - poly(sg.link_ol(ol[I]))

sigma = data.sigma
plt.figure(figsize=(5,4))
plt.plot(ol[I], res_4, '.', color=colors[o-orders[0]])
plt.show()

Analysing residuum around polynomial for each order

In [ ]:
o =29 # fell free to play with it...
n =  6 # order of polynomial
bins_abs = np.linspace(-1, 1, 21)
bins_rel = np.linspace(-5, 5, 21)
for o in orders:
    I = data.index_order(o)
    sig = np.sqrt(0.01 + data.sigma**2)
    p = np.polynomial.Polynomial.fit(sg.link_ol(ol[I]), data.x[I], n, w=1./sig[I])
    res = (data.x[I] - p(sg.link_ol(ol[I])))
    #pd.cut(res, np.linspace(-4,4,5))
    #res.hist(bins=10)
    plt.figure(figsize=(5,2))
    plt.subplot(131)
    plt.title('abs '+str(o))
    plt.hist(res, bins=bins_abs, range=(-0.5, 0.5))
    plt.subplot(132)
    plt.title('rel ' + str(o))
    plt.hist(res/sig[I], bins=bins_rel, range=(-5,5))
    plt.subplot(133)
    plt.plot(data.l[I], res, '.', color=colors[o-orders[0]])
plt.show()

### Outlier removal order by order
criteria: absolut $\epsilon < \sigma_a$ and relative $\epsilon/\sigma < \sigma_r$ 
We convert $\sigma_a$ in errors of radial velocity using the formula
$$
\delta_{rv} = \gamma_o \,\sigma_a,\quad \gamma_o = 3\cdot 10^8 \frac{d\lambda/dx}{\lambda} = \frac{3\cdot 10^{8} }{\lambda |dx/d\lambda|_o}=\frac{3\cdot 10^{8} \cdot o }{\lambda |\partial_{o\lambda}x(o\lambda, o)|}
$$
where $x=x(o\lambda, o)$ is taken in linear global approximation

In [ ]:
sigma_a = 0.5
sigma_r = 3
# global linear approximation
p_lin = np.polynomial.Polynomial.fit(ol, data.x, deg=1)
gamma_o = {o: o * 3e8 / (data.l[data.index_order(o)].mean() * p_lin.coef[1]) for o in orders}
gamma = o * 3e8 / (data.l * p_lin.coef[1])
plt.figure(figsize=(5,4))
for o in orders:
    plt.plot( [o], [gamma_o[o]], 'o', color=colors[o-orders[0]])
plt.xlabel('order')
plt.ylabel('average $\gamma$ factor')
plt.show()

Therefore with a target resolution of $100 m/s$ the reuired precision depending on the order needs to be adjusted 

In [ ]:
delta_vr = np.linspace(0, 200, 50)
Io = np.any([data.index_order(o) for o in orders], axis=0)
plt.figure(figsize=(5,4))
plt.plot(delta_vr, [ sum(gamma[Io] * data.sigma[Io] < d_vr) for d_vr in delta_vr])
plt.title('theoretical precision per data')
plt.xlabel('target rv precision')
plt.ylabel('number of usable spectral lines')
plt.show()

In [ ]:
delta_vr = np.linspace(0, 200, 50)
Io = np.any([data.index_order(o) for o in orders], axis=0)
plt.figure(figsize=(5,4))
for o in orders:
    I = data.index_order(o)
    plt.plot(delta_vr, [ sum(gamma[I] * data.sigma[I] < d_vr) for d_vr in delta_vr], color=colors[o-orders[0]])
plt.title('theoretical precision per data order by order')
plt.xlabel('target rv precision')
plt.ylabel('number of usable spectral lines')
plt.show()

# Two dimensional modeling